```
p_паттерн парсит текст (str, pos), 
    вызывая другие паттерны, возвращающие древовидные структуры
    эти древовидные структуры передает одному из правил
        (остальные вызовы правил закомментированы 
            - в рамках одного текста каждый паттерн имеет ровно 1 смысл)
    и возвращает (pos, результат этого правила), помещенный в массив
    
    /*пока на ошибках парсинга концентрировать не будем*/
    если ничего не удалось распарсить, ???возвращаемый массив будет пустым???
    если удалось распарсить несколько вариантов - в массиве будет насколько вариантов
        сначала парсятся все исключения
        потом парсятся все обычные варианты (если нет исключений)
```

```
r_правило получает список древовидных структур
    обрабатывает их по определенному правилу
    возвращает древовидную структуру
```

```
древовидная структура - map, со следующими ключами
    type: 'noun'/'adj'/'verb'/'num'/... - определяем через isinstance()
    постоянные параметры (для сущ.: род, число)
    переменные параметры (для сущ.: падеж)
    talk: массив древовидных структур
        или пар (тип, слово), где тип - main/punct/other
```

```
str(древовидная структура)
    превращает древовидную структуру в строку
```

```
для каждого типа элемента древовидной структуры будет map
    где каждой строке (для сущ. - слово в и.п.) будет соответствовать
        структура данных, которая вместе в переменными параметрами передается в 
        sh_функция для этого типа элемента древовидной структуры
            которая будет возвращать слово в соотв. форме (для сущ. в соотв. падеже)
```

In [1]:
from classes import *

### Отображение

In [2]:
def show_noun1(st,word,ip,rp,dp,vp,tp,pp):
    if st.pad=='ip' : return word+ip
    if st.pad=='rp' : return word+rp
    if st.pad=='dp' : return word+dp
    if st.pad=='vp' : return word+vp
    if st.pad=='tp' : return word+tp
    if st.pad=='pp' : return word+pp
    raise RuntimeError()


In [3]:
def show_num1(st,word,ends):
    if st.chis=='mn' :
        (ip,rp,dp,vp,tp,pp)=ends['mn']
    else:
        (ip,rp,dp,vp,tp,pp)=ends[st.rod]
        
    if   st.pad=='ip' : rez=ip
    elif st.pad=='rp' : rez=rp
    elif st.pad=='dp' : rez=dp
    elif st.pad=='vp' : rez=vp[0] if st.odush else vp[1]
    elif st.pad=='tp' : rez=tp
    elif st.pad=='pp' : rez=pp
    else: raise RuntimeError()
        
    return word+rez


In [4]:
def show_adj1(st,word,ends):
    if st.chis=='mn' :
        (ip,rp,dp,vp,tp,pp,sh)=ends['mn']
    else:
        (ip,rp,dp,vp,tp,pp,sh)=ends[st.rod]
        
    if   st.pad=='ip' : rez=ip
    elif st.pad=='rp' : rez=rp
    elif st.pad=='dp' : rez=dp
    elif st.pad=='vp' : rez=vp[0] if st.odush else vp[1]
    elif st.pad=='tp' : rez=tp
    elif st.pad=='pp' : rez=pp
    elif st.pad=='sh' : rez=sh
    else: raise RuntimeError()
        
    return word+rez


In [5]:
#show_noun_map['я']=lambda st: show_noun1(st,"","я","меня","мне","меня","мной","мне")


In [6]:
show_adj_map['летучий'] =\
    lambda st: show_adj1(st,'летуч' ,
        {
            'm' :('ий','его','ему',('его','ий'),'им' ,'ем','' ),
            's' :('ее','его','ему',('ий', 'ий'),'им' ,'ем','е'),
            'g' :('ая','ей' ,'ей' ,('ую', 'ую'),'ей' ,'ей','а'),
            'mn':('ие','их' ,'им' ,('их', 'ие'),'ими','их','и'),
        }
    )

In [7]:
show_num_map['один']=\
    lambda st: show_num1(st,'од' ,
        {
            'm' :('ин','ного','ному',('ного','ин'),'ним' ,'ном'),
            's' :('но','ного','ному',('но'  ,'но'),'ним' ,'ном'),
            'g' :('на','ной' ,'ной' ,('ну'  ,'ну'),'ной' ,'ной'),
            'mn':('ни','них' ,'ним' ,('них', 'ни'),'ними','них'),
        }
    )
show_num_map['два']=\
    (lambda st: \
        ("две" if st.rod=='g' else "два")                         if st.pad=='ip' else \
        "двух"                                                    if st.pad=='rp' else \
        "двум"                                                    if st.pad=='dp' else \
        ("двух" if st.odush else "две" if st.rod=='g' else "два") if st.pad=='vp' else \
        "двумя"                                                   if st.pad=='tp' else \
        "двух"                                                    if st.pad=='pp' else \
        throw(RuntimeError('unknown pad: '+st.pad))
    )
show_num_map['три']=\
    lambda st: show_num1(st,'тр' ,
        {   'mn':('и','ёх' ,'ём' ,('ёх', 'и'),'емя','ёх'), }    )
show_num_map['четыре']=\
    lambda st: show_num1(st,'четыр' ,
        {   'mn':('е','ёх' ,'ём' ,('ёх', 'е'),'ьмя','ёх'), }    )
show_num_map['пять']=\
    lambda st: show_num1(st,'пят' ,
        {   'mn':('ь','и' ,'и' ,('ь', 'ь'),'ью','и'), }    )
show_num_map['шесть']=\
    lambda st: show_num1(st,'шест' ,
        {   'mn':('ь','и' ,'и' ,('ь', 'ь'),'ью','и'), }    )

def show_verb1(st,word,end_map):
    if st.form=='neopr' or st.form=='povel':
        end = end_map[st.form]
    else:
        (i,you,he,shi,it,we,yous,they)=end_map[st.form]
        if   (st.pers,st.chis)==(1,'ed'):
            end = i
        elif (st.pers,st.chis)==(2,'ed'):
            end = you
        elif (st.pers,st.chis)==(3,'ed'):
            if   st.rod=='m' : end=he
            elif st.rod=='g' : end=she
            elif st.rod=='s' : end=it
        elif (st.pers,st.chis)==(1,'mn'):
            end = we
        elif (st.pers,st.chis)==(2,'mn'):
            end = yous
        elif (st.pers,st.chis)==(3,'mn'):
            end = they
    return word+end

show_verb_map['видеть']=\
    lambda st: show_verb1(st,'ви',
        {    'neopr':"деть",
             'povel':"дь",
             'nast': ("жу","дишь","дит","дит","дит","дим","дите","дят")
        }   )

### Правила Словарь

In [8]:
ruwords={} #какие-то однословные правила складываются сюда, а какие-то остаются в функциях

def add_runoun(name,och,odush,rod,chis,word,ip,rp,dp,vp,tp,pp):
    ruwords[name]=StNoun(name,och,odush,rod,chis,'ip')
    show_noun_map[name]=lambda st: show_noun1(st,word,ip,rp,dp,vp,tp,pp)

def add_runoun2(name,rod,word,ip,rp,dp,vp,tp,pp,\
               mname,odush,mword,mip,mrp,mdp,mvp,mtp,mpp):
    add_runoun(name,mname,odush,rod,'ed',word,ip,rp,dp,vp,tp,pp)
    add_runoun(mname,name,odush,rod,'mn',mword,mip,mrp,mdp,mvp,mtp,mpp)


In [9]:
ruwords["я (муж)"]=StProNoun("я","мы",1,True,'m','ed','ip')
ruwords["я (жен)"]=StProNoun("я","мы",1,True,'g','ed','ip')
show_noun_map["я"]=lambda st: show_noun1(st,'',"я","меня","мне","меня","мной","мне")

#                                  ип ,нет ,дать,вижу,творю,думаю
add_runoun2("кот"   ,'m'  ,'кот'  ,'' ,'а' ,'у' ,'а' ,'ом' ,'е' ,
            "коты"  ,True ,'кот'  ,'ы','ов','ам','ов','ами','ах')

add_runoun2("волк"   ,'m'  ,'волк'   ,'' ,'а' ,'у' ,'а' ,'ом' ,'е' ,
            "волки"  ,True ,'волк'   ,'и','ов','ам','ов','ами','ах')
add_runoun2("мальчик",'m'  ,'мальчик','' ,'а' ,'у' ,'а' ,'ом' ,'е' ,
            "мальчики",True,'мальчик','и','ов','ам','ов','ами','ах')

add_runoun2("джем"  ,'m'  ,'джем' ,'' ,'а' ,'у' ,''  ,'ом' ,'е' ,
            "джемы" ,False,'джем' ,'ы','ов','ам','ы' ,'ами','ах')

add_runoun2("ящик"  ,'m'  ,'ящик' ,'' ,'а' ,'у' ,''  ,'ом' ,'е' ,
            "ящики" ,False,'ящик' ,'и','ов','ам','и' ,'ами','ах')
add_runoun2("урок"  ,'m'  ,'урок' ,'' ,'а' ,'у' ,''  ,'ом' ,'е' ,
            "уроки" ,False,'урок' ,'и','ов','ам','и' ,'ами','ах')

add_runoun2("кошка" ,'g'  ,'кошк' ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "кошки" ,True ,'кош'  ,'ки','ек','кам','ек','ками','ках')

add_runoun2("белка" ,'g'  ,'белк' ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "белки" ,True ,'бел'  ,'ки','ок','кам','ок','ками','ках')

add_runoun2("собака",'g'  ,'собак','а' ,'и' ,'е'  ,'у'  ,'ой' ,'е' ,
            "собаки",True ,'собак','и' ,''  ,'ам' ,''   ,'ами','ах')

add_runoun2("зебра",'g'   ,'зебр' ,'а' ,'ы' ,'е'  ,'у'  ,'ой' ,'е' ,
            "зебры",True  ,'зебр' ,'ы' ,''  ,'ам' ,''   ,'ами','ах')

add_runoun2("ручка" ,'g'  ,'ручк' ,'а' ,'и' ,'е'  ,'у'  ,'ой' ,'е'  ,
            "ручки" ,False,'руч'  ,'ки','ек','кам','ки','ками','ках')
add_runoun2("чашка" ,'g'  ,'чашк' ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "чашки" ,False,'чаш'  ,'ки','ек','кам','ки','ками','ках')

add_runoun2("кепка"  ,'g'  ,'кепк'  ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "кепки"  ,False,'кеп'   ,'ки','ок','кам','ки','ками','ках')
add_runoun2("шапка"  ,'g'  ,'шапк'  ,'а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "шапки"  ,False,'шап'   ,'ки','ок','кам','ки','ками','ках')
add_runoun2("коробка",'g'  ,'коробк','а' ,'и' ,'е'  ,'у' ,'ой'  ,'е'  ,
            "коробки",False,'короб' ,'ки','ок','кам','ки','ками','ках')

add_runoun2("крыса" ,'g'  ,'крыс' ,'а','ы','е' ,'у' ,'ой' ,'е' ,
            "крысы" ,True ,'крыс' ,'ы','' ,'ам',''  ,'ами','ах')
add_runoun2("курица",'g'  ,'куриц','а','ы','е' ,'у' ,'ой' ,'е' ,
            "курицы",True ,'куриц','ы','' ,'ам',''  ,'ами','ах')
add_runoun2("лиса"  ,'g'  ,'лис'  ,'а','ы','е' ,'у' ,'ой' ,'е' ,
            "лисы"  ,True ,'лис'  ,'ы','' ,'ам',''  ,'ами','ах')
    
add_runoun2("шляпа" ,'g'  ,'шляп' ,'а','ы','е' ,'у' ,'ой' ,'е' ,
            "шляпы" ,False,'шляп' ,'ы','' ,'ам','ы' ,'ами','ах')
add_runoun2("ваза"  ,'g'  ,'ваз'  ,'а','ы','е' ,'у' ,'ой' ,'е' ,
            "вазы"  ,False,'ваз'  ,'ы','' ,'ам','ы' ,'ами','ах')
add_runoun2("лампа" ,'g'  ,'ламп' ,'а','ы','е' ,'у' ,'ой' ,'е' ,
            "лампы" ,False,'ламп' ,'ы','' ,'ам','ы' ,'ами','ах')
add_runoun2("звезда",'g'  ,'звезд','а','ы','е' ,'у' ,'ой' ,'е' ,
            "звёзды",False,'звёзд','ы','' ,'ам','ы' ,'ами','ах')

add_runoun2("свинья",'g'  ,'свинь','я' ,'и' ,'е'  ,'ю' ,'ёй'  ,'е'  ,
            "свиньи",True ,'свин' ,'ьи','ей','ьям','ей','ьями','ьях')

add_runoun2("мышь"   ,'g'  ,'мыш'   ,'ь','и' ,'и' ,'ь' ,'ью' ,'и' ,
            "мыши"   ,True ,'мыш'   ,'и','ей','ам','ей','ами','ах')

add_runoun2("кровать",'g'  ,'кроват','ь','и' ,'и' ,'ь' ,'ью' ,'и' ,
            "кровати",False,'кроват','и','ей','ям','и' ,'ями','ях')

add_runoun2("ружьё"  ,'s'  ,'ружь'  ,'ё' ,'я' ,'ю'  ,'ё' ,'ём'  ,'е'  ,
            "ружья"  ,False,'руж'   ,'ья','ей','ьям','ьи','ьями','ьях')

add_runoun2("вареньё",'s'  ,'варень','е' ,'я' ,'ю'  ,'е' ,'ем' ,'е' ,
            "варенья",False,'варен' ,'ья','ий','ьям','ья','ьями','ьях')

add_runoun2("ребёнок",'m'  ,'ребён','ок' ,'ка' ,'ку'  ,'ка' ,'ком' ,'ке' ,
            "дети"   ,True ,'дет'  ,'и'  ,'ей' ,'ям'  ,'ей' ,'ьми' ,'ях' )

add_runoun2("котёнок",'m'  ,'котён','ок' ,'ка' ,'ку'  ,'ка' ,'ком' ,'ке' ,
            "котята" ,True ,'котят','а'  ,''   ,'ам'  ,''   ,'ами' ,'ах')

add_runoun('часы (предмет)',None,False,'m','mn','час','ы','ов','ам','ы','ами','ах')


In [10]:
def r_letuchij(): return StAdj('летучий',True,'g','ed','ip')

def r_odin(): return StNum('один','1',False,'m','ed','ip')
def r_dva(): return StNum('два','2-4',False,'m','mn','ip')
def r_tri(): return StNum('три','2-4',False,'m','mn','ip')
def r_chetyre(): return StNum('четыре','2-4',False,'m','mn','ip')
def r_pyat(): return StNum('пять','>=5',False,'m','mn','ip')
def r_shest(): return StNum('шесть','>=5',False,'m','mn','ip')

#(self,word,oasp=0,asp=None,form=None,rod=0,chis=0,pers=0)
def r_videt(): return StVerb('видеть',"увидеть",'nesov','neopr',None,None,None)

### Паттерны Словарь

### Правила Составные

In [11]:
def M(obj,**kvargs):
    for k,v in kvargs.items():
        setattr(obj,k,v)
    return obj


In [12]:
def r_noun_and_noun(sn,a,n):
    return StNoun([
        I(dep=sn),
        I(nodep=S('и',a.attrs)),
        I(dep=n)
    ],c='mn', p='ip',o=False,r='m')
def r_noun_comma_noun(sn,c,n):
    return StNoun([
        I(dep=sn),
        I(punct=S(',',c.attrs)),
        I(dep=n)
    ],c='mn', p='ip',o=False,r='m')


In [13]:
def r_a_noun(a,n): 
    return SAttrs.to_right(a,n)

def r_adj_noun(a,n): 
    return StNoun([
        I(dep=a),
        I(maindep=n)
    ])


In [14]:
def r_numeral_noun(num,n):
    if num.chis!=n.chis :
        print('не совпадают числа числ. и сущ.:',str(num),str(n))
    return StNum([
        I(quantity=num,
            chis=n.chis,
            rod=n.rod,
            odush=n.odush),
        I(maindep=n)
    ],quantity=num.quantity)

def r_noun_numeral(n,num):
    return StNoun([
        I(maindep=n),
        I(nomer=num)
    ])


In [15]:
def r_to_verb(t,v): 
    return SAttrs.to_right(t,v)

def r_noun_verb(n,v): 
    return StVerb([
        I(ip=n),
        I(main=v,
            form='nast',
            pers=n.pers,
            chis=n.chis,
            rod=n.rod)
    ])
def r_verb_noun(v,n): 
    return StVerb([
        I(maindep=v),
        I(ip=n,
            pad='vp')
    ])

In [16]:
dict_adj={
    'a' : S(''),
    #'a' : r_nekotoryj(),
    'an' : S(''),
    #'an' : r_nekotoryj(),
}

In [17]:
dict_noun={
    #'cat':   ruwords["кошка"],
    'cat':   ruwords["кот"],
    #'cats':   ruwords["кошки"],
    'cats':   ruwords["коты"],
    
    #'rat':   ruwords["мышь"],
    'rat':   ruwords['крыса'],
    #'rats':   ruwords["мыши"],
    'rats':   ruwords['крысы'],
    
    'bat':   r_adj_noun(deepcopy(r_letuchij()),deepcopy(ruwords["мышь"])),
    'bats':   r_adj_noun(deepcopy(r_letuchij()),deepcopy(ruwords["мыши"])),
    
    'lesson':ruwords["урок"],
    'lessons':ruwords["уроки"],
    
    'cap':   ruwords["кепка"],
    #'cap':  ruwords["шапка"],
    'caps':   ruwords["кепки"],
    #'caps':  ruwords["шапки"],
    
    'pen':   ruwords["ручка"],
    'pens':   ruwords["ручки"],
    
    'hat':   ruwords["шляпа"],
    'hats':   ruwords["шляпы"],
    
    'hen':   ruwords["курица"],
    'hens':   ruwords["курицы"],
    
    'dog':   ruwords['собака'],
    'dogs':   ruwords['собаки'],
    'pig':   ruwords['свинья'],
    'pigs':   ruwords['свиньи'],
    'gun':   ruwords['ружьё'],
    'guns':   ruwords['ружья'],
    'cup':   ruwords['чашка'],
    'cups':   ruwords['чашки'],
    
    #'box':   ruwords['коробка'],
    #'boxes':   ruwords['коробки'],
    'box':   ruwords['ящик'],
    'boxes':   ruwords['ящики'],

    'jam':   ruwords['джем'],
    #'jam':   ruwords['варенье'],

    'bed':   ruwords['кровать'],
    'beds':   ruwords['кровати'],

    'fox':   ruwords['лиса'],
    'foxes':   ruwords['лисы'],
    
    'child': ruwords['ребёнок'],
    'children': ruwords['дети'],

    'kitten': ruwords['котёнок'],
    'kittens': ruwords['котята'],
    'vase': ruwords['ваза'],
    'vases': ruwords['вазы'],
    'star': ruwords['звезда'],
    'stars': ruwords['звёзды'],
    'lamp': ruwords['лампа'],
    'lamps': ruwords['лампы'],
    
    'squirrel': ruwords['белка'],
    'squirrels': ruwords['белки'],
    'wolf': ruwords['волк'],
    'wolfs': ruwords['волки'],
    'zebra': ruwords['зебра'],
    'zebras': ruwords['зебры'],
    'boy': ruwords['мальчик'],
    'boys': ruwords['мальчики'],
#    'boy': ruwords['парень'],
#    'boy': ruwords['парни'],

    'watch': ruwords['часы (предмет)']
}

In [18]:
dict_pronoun_ip={
    'I': ruwords["я (муж)"],
    #'I': ruwords["я (жен)"],
}
dict_pronoun_vp={
    'me': ruwords["я (муж)"],
    #'I': ruwords["я (жен)"],
}


In [19]:
dict_numeral={
    'one':   r_odin(),
    'two':   r_dva(),
    'three': r_tri(),
    'four':  r_chetyre(),
    'five':  r_pyat(),
    'six':   r_shest(),
}

In [20]:
dict_verb={}
dict_verb_s={}

dict_verb  ['see']= r_videt()
dict_verb_s['sees']= r_videt()


### Паттерны Составные

In [21]:
def debug_pp(fun):
    def wrapper(s,p):
        print('{'+'.'*p+fun.__name__)
        rezs=fun(s,p)
        print('_'+'.'*p+str(len(rezs)),'in ',fun.__name__,'}',
              [(p,str(r)) for (p,r) in rezs],'\n')
        for i in rezs:
            if isinstance(i[1],StDeclinable):
                i[1].check_attrs('wrapper:'+fun.__name__)
        return rezs
#    return wrapper
    return fun

In [22]:
@debug_pp
def p_numeral(s,p):
    return fromdict(dict_numeral)(s,p)

In [23]:
#2->
@debug_pp
def p_adj(s,p):
    return fromdict(dict_adj)(s,p)

In [24]:
@debug_pp
def p_adj_noun3(s,p):
    rezs=[]
    if 1: #исключение словосочетания
        def lp_an_a(str,pos):
            lrezs=word('an')(str,pos)
            if len(lrezs)>0 : return lrezs
            return word('a')(str,pos)
        rezs+=p_seq(s,p,[ lp_an_a, p_noun3 ],r_a_noun)
        #return p_seq(s,p,[ lp_an_a, p_noun3 ],r_nekotoryj_noun)
    if len(rezs)>0 : return rezs
    #словосочетание
    return p_seq(s,p,[ p_adj, p_noun3 ],r_adj_noun)    
    
@debug_pp
def p_noun3(s,p):
    rezs=  p_adj_noun3(s,p)
    rezs+= p_numeral(s,p)
    rezs+= fromdict(dict_noun)(s,p)
    rezs+= fromdict(dict_pronoun_ip)(s,p)
    if len(rezs)>0 : return rezs
    rezs+= fromdict(dict_pronoun_vp)(s,p)
    return rezs

@debug_pp
def p_noun2(s,p):
    rezs=  p_seq(s,p,[ p_noun3, p_numeral ], r_noun_numeral)
    rezs+= p_noun3(s,p)
    return rezs

@debug_pp
def p_noun1(s,p):
    rezs=  p_seq(s,p,[ p_numeral, p_noun2 ], r_numeral_noun)
    rezs+= p_noun2(s,p)
    return rezs

#1->
@debug_pp
def p_noun(s,p):
    @debug_pp
    def p_noun1_and_noun(s,p):
        rezs=  p_seq(s,p,[ p_noun1, word('and'), p_noun ],r_noun_and_noun  )
        rezs+= p_seq(s,p,[ p_noun1, word(',')  , p_noun ],r_noun_comma_noun)
        return rezs
    rezs=  p_noun1_and_noun(s,p)
    rezs+= p_noun1(s,p)
    return rezs

In [25]:
@debug_pp
def p_verb1(s,p):
    rezs=  p_seq(s,p,[fromdict(dict_verb),p_noun],r_verb_noun)
#    rezs+= p_seq(s,p,[word('to'),fromdict(dict_verb)],r_to_verb)
    if len(rezs)>0: return rezs
    rezs+= fromdict(dict_verb)(s,p)
    if len(rezs)>0: return rezs
    rezs+= fromdict(dict_verb_s)(s,p)
    return rezs

@debug_pp
def p_verb(s,p):
    rezs=  p_seq(s,p,[p_noun,p_verb1],r_noun_verb)
    rezs+= p_seq(s,p,[word('to'),p_verb1],r_to_verb)
    if len(rezs)>0: return rezs
    rezs+= p_verb1(s,p)
    return rezs

In [26]:
@debug_pp
def p_phrase(s,p):
    rezs=p_verb(s,p)
    if len(rezs)>0: return rezs
    rezs=p_noun(s,p)
    if len(rezs)>0: return rezs
    rezs=p_adj(s,p)
    return rezs

In [27]:
dict_proper={}
@debug_pp
def p_sentence(s,p):
    first_capital = s[p]=='I' or ch_title in s[p].attrs.changers
    def r_sentence(ph,d):
        rez=StC([I(nodep=ph),I(punct=d)])
        if first_capital: rez.attrs.changers|={ch_sentence}
        return rez
    restore_title=False
    if ch_title in s[p].attrs.changers and s[p] not in dict_proper:
        s[p].attrs.changers-={ch_title}
        restore_title=True
    rezs=p_seq(s,p,[p_phrase,word('.')],r_sentence)
    if restore_title:
        s[p].attrs.changers|={ch_title}
    return rezs

In [28]:
def en2ru(s): # main
    def maxlen_rezs(rezs):
        m=0
        im=set()
        for i in range(len(rezs)):
            if rezs[i][0]>m:
                m=rezs[i][0]
                im={i}
            elif rezs[i][0]==m:
                im.add(i)
        return [rezs[i] for i in im]
                
    s=[ i for i in tokenizer(s)]
    if len(s)==0:
        print('no tokens')
        return ''

    p=0
    rez=[]
    while p<len(s):
        rezs=maxlen_rezs(p_sentence(s,p))
        if len(rezs)==0: break
        p1,r1=next(iter(rezs))
        if len(rezs)>1:
            print('multiple results:')
            print(SAttrs.join(s[p:p1]))
            for void,r in rezs:
                print(str(r))
        p=p1
        rez.append(I(nodep=r1))

    if len(rez)>0:
        if p!=len(s):
            print('NOT PARSED:')
            print(SAttrs.join(s[p:]))
        return str(StC(rez))
    else:
        assert p==0
        rezs=maxlen_rezs(p_phrase(s,p))
        if len(rezs)==0:
            raise ParseError('no results')
        p1,r1=next(iter(rezs))
        if len(rezs)>1:
            print('multiple results:')
            print(SAttrs.join(s[p:p1]))
            for void,r in rezs:
                print(str(r))
        p=p1
        if p!=len(s):
            print('NOT PARSED!')
            print(SAttrs.join(s[p:]))
        return str(r1)


# Тесты

```
местоимения вп
    7)  у меня есть
        у меня нет
        покажи
        дай
    8)  скажи
    9)  (находится) на/в
    10) вопросы-ответы
        считать от до
        ловить
    11) сколько
    13) где
    14) какого цвета
        КОНТЕКСТ
    15) посмотри

watch, двое, трое, пятеро
...
I() - чтобы чтобы модификации свойств записывала в 3й элемент тупла
get_property(tuple)
    если 2 элемента - берет настоящее сойство
    если 3 элемента - берет свойство из 3го элемента
использовать её в init-ах для main и maindep
.pull_deferred():
    с каждым tupl-ом из talk
        если он Struct
            вызываем его.pull_deferred()
        применяем свойства из 3го элемента tupl-а
    подтягиваем .pre из talk[0]
.tistr()
    .pull_deferred
    .__str__

сделать кэширование результатов
    статистика использования паттернов/словарей/правил

атрибуты слов: (теги)
...
нужен LR-парсер, причем недетерминированный - посмотреть, какие есть библиотеки
    возможно стоит придумать уровни для паттернов...
```

In [29]:
def decline(s,pads=['ip','rp','dp','vp','tp','pp']):
    s=[ i for i in tokenizer(s)]
    # добавить дочитывание точки и остаточных пробелов
    rezs=[res for pos,res in p_noun(s,0) if pos==len(s)]
    if len(rezs)!=1:
        raise TextError(rezs)
    tmp=rezs[0]
    
    m=[]
    for p in pads:
        #print(str(tmp))
        prompt= \
            '' if p=='ip' else\
            'нет ' if p=='rp' else\
            'дать ' if p=='dp' else\
            'вижу ' if p=='vp' else\
            'творю ' if p=='tp' else\
            'думаю о ' if p=='pp' else\
            throw(ValueError('bad pad: '+p))
        #rez=deepcopy(tmp)
        tmp.pad=p
        m.append(prompt+str(tmp))#        print(prompt+str(tmp))
    return m


In [30]:
decline('one watch')

не совпадают числа числ. и сущ.: один часы
не совпадают числа числ. и сущ.: один часы
не совпадают числа числ. и сущ.: один часы


['одни часы',
 'нет одних часов',
 'дать одним часам',
 'вижу одни часы',
 'творю одними часами',
 'думаю о одних часах']

In [31]:
en2ru('')

no tokens


''

In [32]:
en2ru('I see jam and one cup')

'я вижу джем и одну чашку'